Create Azure SQL Database
============================================

Steps of this procedure include:
1. Set variables and set up Notebook
1. Connect to Azure account and subscription
1. Provision firewall rules
1. Create SQL database resource

## Set variables
These variables are set based on your inputs in the deployment wizard. You can make changes to these variables but be aware of possible validation errors caused by your changes.






## Notebook Setup 

In [ ]:
import sys, os, json, time, string, random, subprocess
def run_command(command, json_decode = True, printOutput = True):
    print(command)
    process = subprocess.Popen(command.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True)
    output, error = process.communicate()
    if process.returncode != 0: 
        print("Process failed %d \n%s" % (process.returncode, error.decode("utf-8")))
        raise Exception()
    if output:
        output = output.decode("utf-8")
        if printOutput:
            print(output)
        try:
            return json.loads(output)
        except:
            return output


## Connecting to your Azure account


In [ ]:
subscriptions = run_command('az account list', printOutput = False)
if azure_sqldb_subscription not in (subscription["id"] for subscription in subscriptions):
    run_command('az login')

## Setting your Azure subscription


In [ ]:
run_command(
    'az account set '
    '--subscription {0}'
    .format(
        azure_sqldb_subscription));

## Create a server firewall rule

This firewall rule will allow you to access your server and database within IP range immediately after it is created.

In [ ]:
if azure_sqldb_enable_firewall_rule == True:
    create_firewall_rule_result = run_command(
        'az sql server firewall-rule create '
        '--start-ip-address {0} '
        '--end-ip-address {1} '
        '--server {2} '
        '--name {3} '
        '--resource-group {4} '
        .format(
            azure_sqldb_ip_start, 
            azure_sqldb_ip_end, 
            azure_sqldb_server_name, 
            azure_sqldb_firewall_name, 
            azure_sqldb_resource_group_name));

## Create Azure SQL Database

Learn more about the different [cost and performance options](https://go.microsoft.com/fwlink/?linkid=842067) and [other additional customizations](https://go.microsoft.com/fwlink/?linkid=2147212) for creating the database

In [ ]:
create_database_result = run_command(
    'az sql db create '
    '--server {0} '
    '--name {1} '
    '--edition GeneralPurpose '
    '--compute-model Serverless '
    '--family Gen5 '
    '--resource-group {2} '
    '--min-capacity 0.5 '
    '--max-size 32GB '
    '--capacity 1 '
    '--collation {3} '
    .format(
        azure_sqldb_server_name, 
        azure_sqldb_database_name, 
        azure_sqldb_resource_group_name, 
        azure_sqldb_collation));